# Klasyfikator klastrów

## IMPORTS

In [ ]:
!pip install comet-ml

In [ ]:
import comet_ml

import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader

import einops
import numpy as np

import matplotlib
from matplotlib import pyplot as plt

In [ ]:
# import os

# from google.colab import userdata
# key = userdata.get('COMET_API_KEY')
# os.environ['COMET_API_KEY'] = key

In [ ]:
def setup_device():
    if torch.cuda.is_available():
        device = torch.device('cuda')
        # Set default tensor type for cuda
        torch.set_default_dtype(torch.float32)
    elif torch.backends.mps.is_available():
        device = torch.device('mps')
    else:
        device = torch.device('cpu')
        # Ensure we're using float32 on CPU
        torch.set_default_dtype(torch.float64)
    return device

device = setup_device()

print(f"Using {device} device")

Using mps device


## Dataset

In [28]:
!pip install datasets

In [29]:
from datasets import load_dataset, DatasetDict, load_from_disk


In [30]:
!ls

INPAINTING_AUTOENC.ipynb      dataset_clusterization.ipynb
VGG16.py                      dataset_rename.ipynb
VGG16_inpating.py             learn_model_per_cluster.ipynb
__pycache__                   losowanie.ipynb
cluster_clasification.ipynb   main.py
clustered_whole_ds_2          model1.pth
create_mask.py                requirements.txt


In [31]:
ds = load_from_disk("clustered_whole_ds_2").with_format("torch")

In [32]:
ds

DatasetDict({
    train: Dataset({
        features: ['title', 'artist', 'date', 'genre', 'style', 'description', 'filename', 'image', 'cluster'],
        num_rows: 82600
    })
    test: Dataset({
        features: ['title', 'artist', 'date', 'genre', 'style', 'description', 'filename', 'image', 'cluster'],
        num_rows: 10325
    })
    valid: Dataset({
        features: ['title', 'artist', 'date', 'genre', 'style', 'description', 'filename', 'image', 'cluster'],
        num_rows: 10325
    })
})

In [36]:
from datasets import DatasetDict

# Create subsets for each cluster
cluster_subsets = {}

for cluster_id in range(0,20):
    # Filter the dataset for the current cluster
    print(int(cluster_id))
    cluster_data = ds.filter(lambda example: example['cluster'] == cluster_id)
    
    # Add to the cluster_subsets dictionary
    cluster_subsets[int(cluster_id)] = cluster_data

# cluster_subsets now contains subsets for 20 clusters

0


Filter: 100%|██████████| 10325/10325 [00:05<00:00, 1992.29 examples/s]


1


Filter: 100%|██████████| 10325/10325 [00:05<00:00, 2001.88 examples/s]


2


...

Filter: 100%|██████████| 10325/10325 [00:05<00:00, 1898.42 examples/s]


18


Filter: 100%|██████████| 10325/10325 [00:05<00:00, 1875.35 examples/s]


19


Filter: 100%|██████████| 10325/10325 [00:05<00:00, 1941.81 examples/s]


In [37]:
print(cluster_subsets)

{0: DatasetDict({
    train: Dataset({
        features: ['title', 'artist', 'date', 'genre', 'style', 'description', 'filename', 'image', 'cluster'],
        num_rows: 2904
    })
    test: Dataset({
        features: ['title', 'artist', 'date', 'genre', 'style', 'description', 'filename', 'image', 'cluster'],
        num_rows: 380
    })
    valid: Dataset({
        features: ['title', 'artist', 'date', 'genre', 'style', 'description', 'filename', 'image', 'cluster'],
        num_rows: 389
    })
}), 1: DatasetDict({
    train: Dataset({
        features: ['title', 'artist', 'date', 'genre', 'style', 'description', 'filename', 'image', 'cluster'],
        num_rows: 3044
    })
    test: Dataset({
        features: ['title', 'artist', 'date', 'genre', 'style', 'description', 'filename', 'image', 'cluster'],
        num_rows: 354
    })
    valid: Dataset({
        features: ['title', 'artist', 'date', 'genre', 'style', 'description', 'filename', 'image', 'cluster'],
        num_rows: 

In [39]:
for cluster_id, subset in cluster_subsets.items():
    subset.set_format("arrow")
    subset.save_to_disk(f"./cluster_{cluster_id}_subset")
    subset.set_format("torch")
    

Saving the dataset (1/1 shards): 100%|██████████| 322/322 [00:00<00:00, 35433.97 examples/s]


In [ ]:
# change if not colab
num_workers = 12
pin_memory = True
batch_size = 32

cluster_dataloaders = {}

for cluster_id in range(0,20):
    subset = cluster_subsets[cluster_id]
    cluster_dataloaders[cluster_id] = {
            "train": DataLoader(
                subset["train"],
                batch_size=batch_size,
                num_workers=num_workers,
                shuffle=True,
                pin_memory=pin_memory,
            ),
            "test": DataLoader(
                subset["test"],
                batch_size=batch_size,
                num_workers=num_workers,
                shuffle=False,
                pin_memory=pin_memory,
            ),
            "valid": DataLoader(
                subset["valid"],
                batch_size=batch_size,
                num_workers=num_workers,
                shuffle=False,
                pin_memory=pin_memory,
            ),
        }



In [43]:
print(cluster_dataloaders)

{0: {'train': <torch.utils.data.dataloader.DataLoader object at 0x34f6ffa70>, 'test': <torch.utils.data.dataloader.DataLoader object at 0x34ef1fef0>, 'valid': <torch.utils.data.dataloader.DataLoader object at 0x169c60d70>}, 1: {'train': <torch.utils.data.dataloader.DataLoader object at 0x34f6ff3b0>, 'test': <torch.utils.data.dataloader.DataLoader object at 0x168432570>, 'valid': <torch.utils.data.dataloader.DataLoader object at 0x169e061e0>}, 2: {'train': <torch.utils.data.dataloader.DataLoader object at 0x169ef0980>, 'test': <torch.utils.data.dataloader.DataLoader object at 0x169fbaf90>, 'valid': <torch.utils.data.dataloader.DataLoader object at 0x34f04d010>}, 3: {'train': <torch.utils.data.dataloader.DataLoader object at 0x34f04c200>, 'test': <torch.utils.data.dataloader.DataLoader object at 0x34f04f650>, 'valid': <torch.utils.data.dataloader.DataLoader object at 0x34f04c590>}, 4: {'train': <torch.utils.data.dataloader.DataLoader object at 0x34f04ffb0>, 'test': <torch.utils.data.data

## AUTOENCODER

In [ ]:
PATH = "model1.pth"
# model = VGG16Autoencoder()
new_model = torch.load(PATH)
new_model.eval()
new_model.to(device)

/var/folders/rb/07333lmj29n9vlhvb4phkvpm0000gn/T/ipykernel_41190/743672055.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load(PATH)


VGG16Autoencoder(
  (encoder): VGG16EncoderWithSkipConnections(
    (block1): Sequential(
      (0): Conv2d(4, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace=True)
      (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (3): ReLU(inplace=True)
      (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
    (block2): Sequential(
      (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace=True)
      (2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (3): ReLU(inplace=True)
      (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
    (block3): Sequential(
      (0): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace=True)
      (2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (3): ReLU(inplace=True)
      (4): Conv2d(256, 256

In [52]:
from torchvision import models

class PerceptualLoss(nn.Module):
    def __init__(self):
        super(PerceptualLoss, self).__init__()
        self.feature_extractor = models.vgg16(pretrained=True).features.eval()
        self.layers = ['4', '9', '16']
        for param in self.feature_extractor.parameters():
            param.requires_grad = False

    def forward(self, predicted, targets):
        pred_features = self.extract_features(predicted)
        target_features = self.extract_features(targets)

        loss = 0.0
        for pred_feat, target_feat in zip(pred_features, target_features):
            loss += nn.functional.mse_loss(pred_feat, target_feat)
        return loss

    def extract_features(self, x):
        features = []
        for name, layer in self.feature_extractor._modules.items():
            x = layer(x)
            if name in self.layers:
                features.append(x)
        return features


class CombinedLoss(nn.Module):
    def __init__(self, alpha=0.5):
        super(CombinedLoss, self).__init__()
        self.perceptual_loss = PerceptualLoss()
        self.mse_loss = nn.MSELoss(reduction='none')
        self.alpha = alpha

    def forward(self, predicted, targets, masks):
        loss_weights = torch.where(masks == 0.0, torch.tensor(1.0), torch.tensor(10.0))
        loss_weights = loss_weights.unsqueeze(1)  # batch 1 h w
        loss_weights = loss_weights.to(predicted.device)


        mse = self.mse_loss(predicted, targets) * loss_weights
        mse = mse.mean()

        perceptual = self.perceptual_loss(predicted, targets)

        return self.alpha * mse + (1 - self.alpha) * perceptual


In [60]:
from torchinfo import summary
from tqdm import tqdm 
from create_mask import generate_scaled_blob

LR = 3e-5

loss_func = CombinedLoss(alpha=0.7).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=LR)



/Users/baban/Documents/SIUM2/AUTOENC/.venv/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/baban/Documents/SIUM2/AUTOENC/.venv/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /Users/baban/.cache/torch/hub/checkpoints/vgg16-397923af.pth


URLError: <urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1000)>

In [59]:
from create_mask import generate_scaled_blob


for cluster in range(0,20):
    
    PATH = "model1.pth"
    # model = VGG16Autoencoder()
    new_model = torch.load(PATH)
    new_model.eval()
    new_model.to(device)
    
    train_loader, test_loader, val_loader = cluster_dataloaders[cluster]


    # Train and validate
    for epoch in range(num_epochs):

        # Training phase
        model.train()

        for idx, batch in tqdm(enumerate(train_loader), desc=f"TRAIN_{epoch}"):
            optimizer.zero_grad()  # Zero out gradients before each batch

            images = batch["image"].float() / 255.0
            labels = batch["style"]  # Assuming you still have this in your dataset

            # masks with corrupted part of image
            masks = generate_scaled_blob(images.shape, mask_percentage=(1 / 16) * 100).float() / 255.0

            # apply mask
            images_with_mask = images * (1 - masks.unsqueeze(1))

            # add mask as 4th channel
            images_with_mask = torch.cat((images_with_mask, masks.unsqueeze(1)), dim=1)  # batch 4 imgsize imgsize

            inputs = images_with_mask.to(device)
            targets = images.to(device)

            reconstructed_image = model(inputs)
            loss = loss_func(reconstructed_image, targets, masks)
            loss.backward()
            optimizer.step()


        # Validation phase
        model.eval()

        for idx, batch in tqdm(enumerate(val_loader), desc=f"VAL_{epoch}"):

            images = batch["image"].float() / 255.0

            # masks with corrupted part of image
            masks = generate_scaled_blob(images.shape, mask_percentage=(1 / 16) * 100).float() / 255.0

            #apply mask
            images_with_mask = images * (1- masks.unsqueeze(1))

            #add mask as 4th channel
            images_with_mask = torch.cat((images_with_mask, masks.unsqueeze(1)), dim=1) # batch 4 imgsize imgsize

            inputs = images_with_mask.to(device)
            targets = images.to(device)

            # Forward pass through the model
            # Autoencoder directly outputs the reconstructed image
            reconstructed_image = model(inputs)

            loss = loss_func(reconstructed_image, targets, masks)


    # Set the model to evaluation mode
    model.eval()

    print("Start test")
    # Begin the test phase
    for idx, batch in tqdm(enumerate(test_loader), desc=f"TEST_{num_epochs}"):

        # Load and preprocess images
        images = batch["image"].float() / 255.0  # Assuming images are in [0, 255], normalize to [0, 1]
        labels = batch["style"]  # Assuming you still have this in your dataset (though not used here)

        # masks with corrupted part of image
        masks = generate_scaled_blob(images.shape, mask_percentage=(1 / 16) * 100).float() / 255.0

        # apply mask
        images_with_mask = images * (1 - masks.unsqueeze(1))

        # add mask as 4th channel
        images_with_mask = torch.cat((images_with_mask, masks.unsqueeze(1)), dim=1)  # batch 4 imgsize imgsize
        
        inputs = images_with_mask.to(device)
        targets = images.to(device)
        
        reconstructed_image = model(inputs)

        loss = loss_func(reconstructed_image, targets, masks)
        
        
        print("Save model dict")
        torch.save(model.state_dict(), f"./cluster_{cluster_id}_submodel_dict.pth")
        print("Save model")
        torch.save(model, f"./cluster_{cluster_id}_submodel.pth")
                
            


/var/folders/rb/07333lmj29n9vlhvb4phkvpm0000gn/T/ipykernel_41190/681371507.py:8: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  new_model = torch.load(PATH)
TRAIN_0: 0it [00:

NameError: name 'optimizer' is not defined